## Homework: Vector Search

### Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [2]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
v

array([ 7.82226324e-02, -4.01311740e-02,  3.86136174e-02, -1.78962495e-04,
        8.92346352e-02, -5.04591241e-02, -1.05026821e-02,  3.71056013e-02,
       -4.18714099e-02,  3.48084830e-02, -1.20702032e-02, -2.36942321e-02,
        3.87899838e-02,  1.60988197e-02,  3.50747332e-02,  3.04750563e-03,
        5.79672605e-02, -4.10627462e-02, -3.41552794e-02, -2.56396197e-02,
       -3.55263986e-02,  1.42908050e-02, -1.62800327e-02,  3.21446396e-02,
       -4.66897562e-02,  7.89185837e-02,  4.90160957e-02,  1.56760979e-02,
       -1.69109963e-02,  2.26482376e-02,  5.60206138e-02, -3.98361422e-02,
        6.77409470e-02, -1.20210098e-02,  1.12624257e-03, -1.94394384e-02,
       -2.65951455e-02,  1.06177898e-02,  1.69687625e-02,  1.13488035e-02,
       -2.97063123e-02,  5.25258631e-02, -1.41453585e-02,  4.61699925e-02,
        1.17066270e-02, -2.38052979e-02, -6.32557869e-02, -1.92042049e-02,
       -7.10596470e-03,  3.24167833e-02,  2.49618031e-02, -5.27501525e-03,
        2.01149546e-02, -

#### Preparing the documents

In [3]:
import requests

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/documents-with-ids.json"
docs_url = f"{base_url}/{relative_url}?raw=1"
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [5]:
filtered_documents = [
    doc for doc in documents if doc["course"] == "machine-learning-zoomcamp"
]

In [6]:
len(filtered_documents)

375

### Q2. Creating the embeddings

In [7]:
from tqdm.auto import tqdm

embeddings = []

for document in tqdm(filtered_documents):
    qa_text = f"{document['question']} {document['text']}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

  0%|          | 0/375 [00:00<?, ?it/s]

In [8]:
import numpy as np

X = np.array(embeddings)

In [9]:
X.shape

(375, 768)

### Q3. Search
We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix `X` by the vector `v`:

In [10]:
scores = X.dot(v)
max(scores)

0.6506573

In [11]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'You can find the latest and up-to-date deadlines here: https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml\nAlso, take note of Announcements from @Au-Tomator for any extensions or other news. Or, the form may also show the updated deadline, if Instructor(s) has updated it.',
  'section': 'General course-related questions',
  'question': 'Homework - What are homework and project deadlines?',
  'course': 'data-engineering-zoomcamp',
  'id': 'a1daf537'},
 {'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning i

### Q4. Hit-rate for our search engine

In [12]:
import pandas as pd

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/ground-truth-data.csv"
ground_truth_url = f"{base_url}/{relative_url}?raw=1"

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == "machine-learning-zoomcamp"]
ground_truth = df_ground_truth.to_dict(orient="records")

In [13]:
ground_truth

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Does this course have a GitHub repository for the sign-up link?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'How can I structure my questions and answers for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Are the course videos live or pre-recorded?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'When can I start watching the course videos?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'Are the live office hours sessions recorded?',
  'cours

In [14]:
search_engine = VectorSearchEngine(filtered_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [15]:
def hit_rate(relevance_total):
    cnt = 0
    for point in relevance_total:
        cnt += sum(point)
    return cnt / len(relevance_total)

In [16]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q["document"]
    question = q["question"]
    v = embedding_model.encode(question)
    results = search_engine.search(v, num_results=5)
    relevance = [d["id"] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [17]:
hit_rate(relevance_total)

0.9426229508196722

### Q5. Indexing with Elasticsearch

In [18]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine",
            },
        }
    },
}
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [19]:
filtered_documents[1]

{'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
 'section': 'General course-related questions',
 'question': 'Is it going to be live? When?',
 'course': 'machine-learning-zoomcamp',
 'id': '39fda9f0'}

In [20]:
# Add question text vector
for idx, document in enumerate(filtered_documents):
    qv_vector = X[idx]
    document["question_text_vector"] = qv_vector

In [21]:
filtered_documents[1]

{'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
 'section': 'General course-related questions',
 'question': 'Is it going to be live? When?',
 'course': 'machine-learning-zoomcamp',
 'id': '39fda9f0',
 'question_text_vector': array([ 7.06691518e-02, -1.93056501e-02,  7.73837045e-02,  2.37204917e-02,
         8.63914639e-02, -2.04659086e-02,  9.15530790e-03,  4.65581007e-02,
        -4.78189858e-03, -1.43747507e-02, -1.97508316e-02, -4.51360084e-02,
         3.47972400e-02, -4.37994339e-02,  3.87682542e-02, -2.01996807e-02,
        -1.13333564e-03, -4.24227081e-02,  3.04052774e-02, -3.29325497e-02,
        -8.21834244e-03,  2.65473756e-03,  3.13991383e-02,  4.64935377e-02,
        -4.02437337e-02, 

In [22]:
for document in filtered_documents:
    try:
        es_client.index(index=index_name, document=document)
    except Exception as e:
        print(f"Exception occured: {e}")

In [23]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

knn = {
    "field": "question_text_vector",
    "query_vector": v,
    "k": 5,
    "num_candidates": 1000,
}

search_query = {"knn": knn, "_source": ["text", "section", "question", "course", "id"]}

es_results = es_client.search(index=index_name, body=search_query)

In [24]:
es_results["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'VIDWyJABDDAJZD8o-2AF',
 '_score': 0.8253289,
 '_source': {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'}}

In [25]:
es_results["hits"]["hits"][0]["_source"]["id"]

'ee58a693'

### Q6. Hit-rate for Elasticsearch

In [26]:
relevance_total = []
for q in tqdm(ground_truth):
    doc_id = q["document"]
    question = q["question"]
    v = embedding_model.encode(question)

    knn = {
        "field": "question_text_vector",
        "query_vector": v,
        "k": 5,
        "num_candidates": 1000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"],
    }

    results = es_client.search(index=index_name, body=search_query)
    results = [hit["_source"] for hit in results["hits"]["hits"]]

    relevance = [d["id"] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [27]:
hit_rate(relevance_total)

0.9431693989071038